# ClipCard Agent Test: Policy/Trust & Safety

**Purpose:** This notebook tests whether an AI agent (LLM) can correctly parse, validate, and reason about ClipCards in the Policy/Trust & Safety domain.

It evaluates:
1. Can the agent extract kill criteria from a ClipCard?
2. Can it determine if a given scenario would trigger the criteria?
3. Can it suggest appropriate authority windows for different risk levels?

In [ ]:
import json
import pandas as pd

# Load example ClipCard
try:
    with open('../../examples/test.clipcard.json', 'r') as f:
        test_card = json.load(f)
    print("Loaded test ClipCard:")
    print(json.dumps(test_card, indent=2))
except FileNotFoundError:
    print("No test ClipCard found.")
    test_card = None

### Test 1: Extract Kill Criteria

Can we programmatically parse kill criteria?

In [ ]:
if test_card:
    kill_criteria = test_card.get('kill_criteria', [])
    print(f"Found {len(kill_criteria)} kill criteria:\n")
    for i, criterion in enumerate(kill_criteria, 1):
        print(f"{i}. Condition: {criterion['condition']}")
        print(f"   Action: {criterion['action']}\n")
else:
    print("No ClipCard available for testing.")

### Test 2: Scenario Evaluation

Test if scenarios match kill criteria

In [ ]:
# Define test scenarios for Policy/Trust & Safety
scenarios = [
    {
        "name": "Normal Operation",
        "wrongful_action_rate": 0.05,
        "appeal_rate": 0.02,
        "false_positive_rate": 0.01,
        "expected_trigger": False
    },
    {
        "name": "High Wrongful Actions",
        "wrongful_action_rate": 0.85,
        "appeal_rate": 0.03,
        "false_positive_rate": 0.02,
        "expected_trigger": True
    },
    {
        "name": "High Appeal Rate",
        "wrongful_action_rate": 0.10,
        "appeal_rate": 0.25,
        "false_positive_rate": 0.03,
        "expected_trigger": True
    },
    {
        "name": "High False Positives",
        "wrongful_action_rate": 0.08,
        "appeal_rate": 0.04,
        "false_positive_rate": 0.15,
        "expected_trigger": True
    }
]

print("Testing scenarios against typical Policy/TS kill criteria:\n")
print("Assumed thresholds:")
print("  - Wrongful action rate: >= 0.8")
print("  - Appeal rate: > 0.2")
print("  - False positive rate: > 0.1\n")

results = []
for scenario in scenarios:
    # Evaluate against typical Policy/TS thresholds
    wrongful_trigger = scenario['wrongful_action_rate'] >= 0.8
    appeal_trigger = scenario['appeal_rate'] > 0.2
    fp_trigger = scenario['false_positive_rate'] > 0.1
    
    any_trigger = wrongful_trigger or appeal_trigger or fp_trigger
    match = any_trigger == scenario['expected_trigger']
    
    results.append({
        'Scenario': scenario['name'],
        'Wrongful Trigger': wrongful_trigger,
        'Appeal Trigger': appeal_trigger,
        'FP Trigger': fp_trigger,
        'Expected': scenario['expected_trigger'],
        'Match': '✓' if match else '✗'
    })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))
print(f"\nTest Accuracy: {df_results['Match'].value_counts().get('✓', 0)}/{len(results)}")

### Test 3: Authority Window Recommendations

Generate appropriate authority windows based on risk factors

In [ ]:
def recommend_authority_window(impact, uncertainty, reversibility):
    """
    Recommend authority window based on ClipCard risk factors
    
    Args:
        impact: 1-5 scale
        uncertainty: 1-5 scale
        reversibility: 1-5 scale (1=easy to reverse, 5=irreversible)
    """
    risk_score = impact * uncertainty
    
    if risk_score >= 20 or reversibility >= 5:
        return {
            "scope_limit": "1% cohort",
            "time_limit": "24h",
            "auto_pause": True,
            "justification": "Very high risk requires minimal exposure"
        }
    elif risk_score >= 15 or reversibility >= 4:
        return {
            "scope_limit": "5% cohort",
            "time_limit": "7d",
            "auto_pause": True,
            "justification": "High risk requires controlled rollout"
        }
    elif risk_score >= 10:
        return {
            "scope_limit": "10% cohort",
            "time_limit": "14d",
            "auto_pause": False,
            "justification": "Medium risk allows moderate exposure"
        }
    else:
        return {
            "scope_limit": "25% cohort",
            "time_limit": "30d",
            "auto_pause": False,
            "justification": "Lower risk permits broader exposure"
        }

# Test with example scenarios
test_cases = [
    {"name": "High-risk policy", "impact": 5, "uncertainty": 4, "reversibility": 5},
    {"name": "Medium-risk enforcement", "impact": 4, "uncertainty": 3, "reversibility": 2},
    {"name": "Low-risk filter", "impact": 2, "uncertainty": 2, "reversibility": 1}
]

print("Authority Window Recommendations:\n")
for case in test_cases:
    rec = recommend_authority_window(case['impact'], case['uncertainty'], case['reversibility'])
    print(f"{case['name']} (I={case['impact']}, U={case['uncertainty']}, R={case['reversibility']}):")
    print(f"  Scope: {rec['scope_limit']}")
    print(f"  Duration: {rec['time_limit']}")
    print(f"  Auto-pause: {rec['auto_pause']}")
    print(f"  Reason: {rec['justification']}\n")

### Summary

This notebook demonstrates that ClipCard data structures can be:
- Programmatically parsed
- Evaluated against scenarios
- Used to generate risk-appropriate recommendations

This enables AI agents to assist with ClipCard creation, validation, and decision support.